In [1]:
# Load necessary packages
import pandas as pd
import plotly.express as px        # high-level plotly module
import plotly.graph_objects as go  # lower-level plotly module with more functions
import pandas_datareader as pdr    # we are grabbing the data and wb functions from the package
import datetime as dt              # for time and date
import requests                    # api module
#from ipywidgets import widgets     # interactive graphs

import dash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output, State
import plotly.io as pio

In [2]:
today = dt.datetime.now().strftime('%B %d, %Y')  # today's date. this will be useful when sourcing results 

In [3]:
url = 'https://api.covidtracking.com/v1/states/daily.json'
    
response = requests.get(url)

if response.status_code == 200: 
    print('Download successful')
    covid = pd.read_json(response.text)  # convert to dataframe
elif response.status_code == 301: 
    print('The server redirected to a different endpoint.')
elif response.status_code == 401: 
    print('Bad request.')
elif response.status_code == 401: 
    print('Authentication required.')
elif response.status_code == 403: 
    print('Access denied.')
elif response.status_code == 404: 
    print('Resource not found')
else: 
    print('Server busy')

Download successful


In [6]:
covid['pos_rate'] = 100*(covid['positiveTestsViral']+covid['positiveTestsAntigen'])/(covid['totalTestsPeopleViral']+ covid['totalTestsAntigen'])
covid['date'] = pd.to_datetime(covid['date'], format='%Y%m%d')
covid['month'] = covid['date'].dt.strftime('%B %Y') 

date state  positive  probableCases   negative  pending  \
0 2020-10-25    AK   14064.0            NaN   545905.0      NaN   
1 2020-10-25    AL  184355.0        26474.0  1144785.0      NaN   
2 2020-10-25    AR  106115.0         7235.0  1189120.0      NaN   
3 2020-10-25    AS       0.0            NaN     1616.0      NaN   
4 2020-10-25    AZ  238163.0         5462.0  1473706.0      NaN   

  totalTestResultsSource  totalTestResults  hospitalizedCurrently  \
0        totalTestsViral          559969.0                   58.0   
1        totalTestsViral         1302666.0                  922.0   
2        totalTestsViral         1288000.0                  618.0   
3        totalTestsViral            1616.0                    NaN   
4  totalTestsPeopleViral         1706407.0                  828.0   

   hospitalizedCumulative  ...  hospitalizedIncrease  \
0                     NaN  ...                     0   
1                 19595.0  ...                     0   
2                  6718.0  ...                    11   
3                     NaN  ...                     0   
4                 21086.0  ...                    43   

                                       hash  commercialScore  \
0  175a4f70a15c098ac277246bff6ae33838f66029                0   
1  14421edcbba78df1217731644f92a3a03211c44d                0   
2  813725c51fabd6f6078ad953d35346c5d5987dc1                0   
3  3c89978d48f3a4f830660f7149129437b5808f1e                0   
4  7b19d3005937c11aaefe84f6c137e0ae59da6655                0   

   negativeRegularScore  negativeScore positiveScore score grade pos_rate  \
0                     0              0             0     0            NaN   
1                     0              0             0     0            NaN   
2                     0              0             0     0            NaN   
3                     0              0             0     0            NaN   
4                     0              0             0     0            NaN   

          month  
0  October 2020  
1  October 2020  
2  October 2020  
3  October 2020  
4  October 2020  

[5 rows x 57 columns]

In [7]:
covid.drop(covid[covid['state'] == "AS"].index, inplace=True)         # Drop American Samoa
covid.drop(covid[covid['date'] < '2020-03-01'].index, inplace=True)   # Drop January and February when there are few cases

months = covid['month'].unique().tolist()
months.reverse()

In [10]:
# Prepare for Dash
df = covid[['date','state','positive','hospitalizedCurrently','deathIncrease','death']]
df2 = df.copy()
df2.state = 'All States'
df = df.append(df2, ignore_index=True)

In [11]:
app = dash.Dash()

header = html.H1(children="United States Covid-19 Trends (as of " + today + ")")

markdown_text = '''
### Dash and Markdown
The following graphs depict select Covid-19 trends by state. 
Use the filters to choose a subset of states and/or dates to sharpen the analysis. 
Source: The Atlantic [Covid-19 Tracking Project](https://covidtracking.com)
'''
markdown = dcc.Markdown(children=markdown_text)

# Dropdown
dropdown =  html.P([
            html.Label("Select States"),
            dcc.Dropdown(
            id='state-dropdown',
            options=[{'label': i, 'value': i} for i in df['state'].unique().tolist()],
            multi=True,
            searchable= True)
            ], style = {'width' : '80%',
                        'fontSize' : '20px',
                        'padding-left' : '100px',
                        'display': 'inline-block'})
    
# range slider
slider =    html.P([
            html.Label("Select Months"),
            dcc.RangeSlider(id = 'slider',
                        marks = {i : months[i] for i in range(0, len(months))},
                        min = 0,
                        max = len(months)-1,
                        value = [0, len(months)-1])
            ], style = {'width' : '80%',
                        'fontSize' : '20px',
                        'padding-left' : '100px',
                        'display': 'inline-block'})
    
graph1 = dcc.Graph(id="positive", style={'display': 'inline-block'})
graph2 = dcc.Graph(id="curhospital", style={'display': 'inline-block'})
graph3 = dcc.Graph(id="newdeaths", style={'display': 'inline-block'})
graph4 = dcc.Graph(id="totdeaths", style={'display': 'inline-block'})

row1 = html.Div(children=[graph1, graph2])
row2 = html.Div(children=[graph3, graph4])

layout = html.Div(children=[header, markdown, dropdown, slider, row1, row2], style={"text-align": "center"})
app.layout = layout

In [12]:
@app.callback(
    Output('positive', 'figure'),
    [Input('state-dropdown', 'value')],
    [Input('slider', 'value')])
    
# STEP 5: Update Figure
def update_figure(state_values,month_values):

    if state_values is None:
        dff = df.copy()
        dff.drop(dff[dff['state'] == "All States"].index, inplace=True)
        dff = dff.pivot(index='date',columns='state',values='positive')        
    elif state_values[0]=="All States":
        dff = df.copy()
        dff.drop(dff[dff['state'] == "All States"].index, inplace=True)
        dff = dff.pivot(index='date',columns='state',values='positive')
    else:
        if not isinstance(state_values, list): state_values = [state_values]
        temp = df.loc[df['state'].isin(state_values)]
        dff = temp.pivot(index='date',columns='state',values='positive')
    
    # Filter by months
    dff = dff[(dff.index >= dt.datetime.strptime(months[month_values[0]],"%B %Y")) & (dff.index <= dt.datetime.strptime(months[month_values[1]],"%B %Y"))]
    
    fig = go.Figure()
    for column in dff.columns.to_list():
        fig.add_trace(
            go.Scatter(
                x = dff.index,
                y = dff[column],
                name = column,
                mode='lines',
                opacity=0.8,
                hovertemplate = '<extra></extra>State: ' + column + '<br>Date: ' + dff.index.strftime('%m/%d') +'<br>Value: %{y:,}'
            )
        )
      
    # Update remaining layout properties
    fig.update_layout(
        title={
                'text': "New Cases",
                'x':0.5,'xanchor': 'center',
                'font':{'size': 18}
                },
        hovermode='closest',plot_bgcolor='rgba(0,0,0,0)',
        hoverlabel=dict(
            bgcolor = 'white',
            font_size=16),
        width=700,   
        xaxis=dict(
            title="Date",
            zeroline=True,
            showgrid=False,  # Removes X-axis grid lines 
            ),
        yaxis=dict(
            zeroline=True, 
            showgrid=False  # Removes Y-axis grid lines
            )
    )
        
    fig.update_xaxes(showline=True, linewidth=2, linecolor='black')
    fig.update_yaxes(showline=True, linewidth=2, linecolor='black')
    return fig

In [13]:
@app.callback(
    Output('curhospital', 'figure'),
    [Input('state-dropdown', 'value')],
    [Input('slider', 'value')])
    
# STEP 5: Update Figure
def update_figure(state_values,month_values):

    if state_values is None:
        dff = df.copy()
        dff.drop(dff[dff['state'] == "All States"].index, inplace=True)
        dff = dff.pivot(index='date',columns='state',values='hospitalizedCurrently')        
    elif state_values[0]=="All States":
        dff = df.copy()
        dff.drop(dff[dff['state'] == "All States"].index, inplace=True)
        dff = dff.pivot(index='date',columns='state',values='hospitalizedCurrently')
    else:
        if not isinstance(state_values, list): state_values = [state_values]
        temp = df.loc[df['state'].isin(state_values)]
        dff = temp.pivot(index='date',columns='state',values='hospitalizedCurrently')
    
    # Filter by months
    dff = dff[(dff.index >= dt.datetime.strptime(months[month_values[0]],"%B %Y")) & (dff.index <= dt.datetime.strptime(months[month_values[1]],"%B %Y"))]
    
    fig = go.Figure()
    for column in dff.columns.to_list():
        fig.add_trace(
            go.Scatter(
                x = dff.index,
                y = dff[column],
                name = column,
                mode='lines',
                opacity=0.8,
                hovertemplate = '<extra></extra>State: ' + column + '<br>Date: ' + dff.index.strftime('%m/%d') +'<br>Value: %{y:,}'
            )
        )
      
    # Update remaining layout properties
    fig.update_layout(
        title={
                'text': "Current Hospitalizations",
                'x':0.5,'xanchor': 'center',
                'font':{'size': 18}
                },
        hovermode='closest',plot_bgcolor='rgba(0,0,0,0)',
        hoverlabel=dict(
            bgcolor = 'white',
            font_size=16),
        width=700,  
        xaxis=dict(
            title="Date",
            zeroline=True,
            showgrid=False,  # Removes X-axis grid lines 
            ),
        yaxis=dict(
            zeroline=True, 
            showgrid=False  # Removes Y-axis grid lines
            )
    )
        
    fig.update_xaxes(showline=True, linewidth=2, linecolor='black')
    fig.update_yaxes(showline=True, linewidth=2, linecolor='black')
    return fig

In [14]:
@app.callback(
    Output('newdeaths', 'figure'),
    [Input('state-dropdown', 'value')],
    [Input('slider', 'value')])
    
# STEP 5: Update Figure
def update_figure(state_values,month_values):

    if state_values is None:
        dff = df.copy()
        dff.drop(dff[dff['state'] == "All States"].index, inplace=True)
        dff = dff.pivot(index='date',columns='state',values='deathIncrease')        
    elif state_values[0]=="All States":
        dff = df.copy()
        dff.drop(dff[dff['state'] == "All States"].index, inplace=True)
        dff = dff.pivot(index='date',columns='state',values='deathIncrease')
    else:
        if not isinstance(state_values, list): state_values = [state_values]
        temp = df.loc[df['state'].isin(state_values)]
        dff = temp.pivot(index='date',columns='state',values='deathIncrease')
    
    # Filter by months
    dff = dff[(dff.index >= dt.datetime.strptime(months[month_values[0]],"%B %Y")) & (dff.index <= dt.datetime.strptime(months[month_values[1]],"%B %Y"))]
    
    fig = go.Figure()
    for column in dff.columns.to_list():
        fig.add_trace(
            go.Scatter(
                x = dff.index,
                y = dff[column],
                name = column,
                mode='lines',
                opacity=0.8,
                hovertemplate = '<extra></extra>State: ' + column + '<br>Date: ' + dff.index.strftime('%m/%d') +'<br>Value: %{y:,}'
            )
        )
      
    # Update remaining layout properties
    fig.update_layout(
        title={
                'text': "Daily Deaths",
                'x':0.5,'xanchor': 'center',
                'font':{'size': 18}
                },
        hovermode='closest',plot_bgcolor='rgba(0,0,0,0)',
        hoverlabel=dict(
            bgcolor = 'white',
            font_size=16),
        width=700,  
        xaxis=dict(
            title="Date",
            zeroline=True,
            showgrid=False,  # Removes X-axis grid lines 
            ),
        yaxis=dict(
            zeroline=True, 
            showgrid=False  # Removes Y-axis grid lines
            )
    )
        
    fig.update_xaxes(showline=True, linewidth=2, linecolor='black')
    fig.update_yaxes(showline=True, linewidth=2, linecolor='black')
    return fig

In [15]:
@app.callback(
    Output('totdeaths', 'figure'),
    [Input('state-dropdown', 'value')],
    [Input('slider', 'value')])
    
# STEP 5: Update Figure
def update_figure(state_values,month_values):

    if state_values is None:
        dff = df.copy()
        dff.drop(dff[dff['state'] == "All States"].index, inplace=True)
        dff = dff.pivot(index='date',columns='state',values='death')        
    elif state_values[0]=="All States":
        dff = df.copy()
        dff.drop(dff[dff['state'] == "All States"].index, inplace=True)
        dff = dff.pivot(index='date',columns='state',values='death')
    else:
        if not isinstance(state_values, list): state_values = [state_values]
        temp = df.loc[df['state'].isin(state_values)]
        dff = temp.pivot(index='date',columns='state',values='death')
    
    # Filter by months
    dff = dff[(dff.index >= dt.datetime.strptime(months[month_values[0]],"%B %Y")) & (dff.index <= dt.datetime.strptime(months[month_values[1]],"%B %Y"))]
    
    fig = go.Figure()
    for column in dff.columns.to_list():
        fig.add_trace(
            go.Scatter(
                x = dff.index,
                y = dff[column],
                name = column,
                mode='lines',
                opacity=0.8,
                hovertemplate = '<extra></extra>State: ' + column + '<br>Date: ' + dff.index.strftime('%m/%d') +'<br>Value: %{y:,}'
            )
        )
      
    # Update remaining layout properties
    fig.update_layout(
        title={
                'text': "Total Deaths",
                'x':0.5,'xanchor': 'center',
                'font':{'size': 18}
                },
        hovermode='closest',plot_bgcolor='rgba(0,0,0,0)',
        hoverlabel=dict(
            bgcolor = 'white',
            font_size=16),
        width=700,  
        xaxis=dict(
            title="Date",
            zeroline=True,
            showgrid=False,  # Removes X-axis grid lines 
            ),
        yaxis=dict(
            zeroline=True, 
            showgrid=False  # Removes Y-axis grid lines
            ),
    )
        
    fig.update_xaxes(showline=True, linewidth=2, linecolor='black')
    fig.update_yaxes(showline=True, linewidth=2, linecolor='black')
    return fig

In [16]:
app.run_server(debug=True, use_reloader=False)  # Turn off reloader if inside Jupyter

Dash is running on http://127.0.0.1:8050/

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: on
